### Verify we are connected to the block chain

WAVE makes use of a global blockchain. This command will print out various parameters such as how many peers we have (to learn of new blocks), how fresh our chain is, and what version of the WAVE agent we are running

In [ ]:
 !bw2 status

### Make an entity

This creates a keypair you can use to interact with BOSSWAVE resources.

We can add additional metadata to this entity, including
- Contact information (`-c`)
- Descriptive message (`-m`)
- Expiration (`-e`, defaults to 30 days)

This keypair is how we identify ourselves to BOSSWAVE resources. All BOSSWAVE operations (such as subscribing and publishing) are transparently verified by the routing infrastructure because they have a self-contained authenticating proof. This proof is generated by the BOSSWAVE "agent" running in this container, so you as a user will never have to interact directly with that construction.

In [3]:
!bw2 mke -o myentity.ent -c "Your Name Here" -m "Rise Camp 2017"

 ╔╡127.0.0.1:28589 2.7.6 'Klystron'
 ╚╡peers=7 block=3100522 age=50s
Entity created
Public VK: Vq-J1iDsFmnDKoTWYRatwVs5pTghWyP4EdGmuY2Ta2o=
wrote key to file myentity.ent
Waiting for 1 object(s) to publish
Current BCIP set to 2 confirmation blocks or 20 block timeout
confirming:🔗🔗🔗 (last block genesis was 7 seconds ago)  
Successfully published Entity Vq-J1iDsFmnDKoTWYRatwVs5pTghWyP4EdGmuY2Ta2o=

All objects published


### Gain access to a namespace

You have a unique namespace, the name of which is stored in an environment variable as $NAMEPSACE.

We want to interact with resources on `$NAMESPACE`, but we need to have permission to do so. As we just created your entity above, it should not have permission to interact with the namespace yet:

In [ ]:
!echo Namespace is: $NAMESPACE
!bw2 buildchain -t myentity.ent -u $NAMESPACE/*

The `buildchain` command attempts to build a chain of delegations of trust to see if there is a valid path in the DOT graph between the namespace entity and our own. It inspects the URI given by the `-u` argument to find the VK or alias of the entity responsible for administrating that namespace. Then, it traverses the graph of all DOTS granted by that entity until it finds a path of valid, non-expired DOTs that terminates at the VK for `myentity.ent`. This path is called a "DOT chain". As you can see, we do not have permissions yet.

Now, we need to ask the owner of the namespace for permission to operate on their namespace. We could use our public key (Entity VK) but that is quite unwieldy. Lets first make a human readable alias for our public key.

To get the VK of our entity, look for the "Entity VK" field of the following output

In [ ]:
!bw2 i myentity.ent

In [ ]:
!bw2 mkalias --long "choose.a.unique.alias.here" --b64 "paste.your.vk.here"

Now, using that alias, lets ask for permission to be granted to our entity:

In [ ]:
!./getaccess <your alias here>

We've scripted this interaction for the purposes of the tutorial, but normally, gaining access to a namespace (or subset thereof) would be the outcome of a conversation with the namespace owner or some other individual with the ability to delegate permissions on that namespace.

The output of the above command will output something like 

```
DOT created
Hash:  Us19whkr4WXXrYojB5ytzKuKK_sqfmNh_2UWiTtj3mw=
Wrote dot to file:  .Us19whkr4WXXrYojB5ytzKuKK_sqfmNh_2UWiTtj3mw=.dot
Waiting for 1 object(s) to publish
Current BCIP set to 2 confirmation blocks or 20 block timeout
```

The `hash` represents the DOT object in the blockchain and is inspectable by the `bw2` command, as we observe below

Re-run this next line until "Registry" is marked **valid** which indicates that the grant is globally visible in the Registry smart contract.

In [ ]:
!bw2 i <DOT HASH HERE>

When the registry field is valid, we know the DOT object can be seen by everyone, and is ready to be used in a proof.

### Delegate access

We've already created a couple of entities for you --- `light.ent` and `switch.ent` --- that have permissions on $NAMESPACE

In [8]:
!bw2 i light.ent
!bw2 bc -t light.ent -u $NAMESPACE/s.light/* -v

 ╔╡127.0.0.1:28589 2.7.6 'Klystron'
 ╚╡peers=8 block=3100541 age=39s
┳ Type: Entity key file
┣┳ Entity VK: UVhNd_g1RidNqvne6k5aqjOnBimdnYwBE6CelAx3AvQ=
┃┣ Signature: valid
┃┣ Registry: valid
┃┣ Alias: 
┃┣ Keypair: ok
┃┣ Balances: 
┃┃┣  0 (0x72ca2fbe28a7bdcb293e26be476b1cc57be2199f) 0.000000 Ξ
┃┃┣  1 (0xcee631fd80da03024cd2d1a2a0acc7d94ef9e82c) 0.000000 Ξ
┃┃┣  2 (0x15ea41069343205d5ab4f99a158476c14ea8ef82) 0.000000 Ξ
┃┃┣  3 (0x770dce718df79c50cc8204ce2d9a4248aa99105d) 0.000000 Ξ
┃┃┣  4 (0x5ca52d5f1e83c5b08995d3f3bba0b587044d0be6) 0.000000 Ξ
┃┃┣  5 (0x0498fb009b328073bfdf5dfc3d208d1ea78dbcc5) 0.000000 Ξ
┃┃┣  6 (0xee697c66521302b0dbe36df9954e828b87fc7f76) 0.000000 Ξ
┃┃┣  7 (0x1c66a51451e585c0dbdb97e943f40a8f69244fdc) 0.000000 Ξ
┃┃┣  8 (0xd8803ad3cfa498be7a618ddc274e28827336e50a) 0.000000 Ξ
┃┃┣  9 (0x065a7049b18d79b81bbf1958f62d6784db9194aa) 0.000000 Ξ
┃┃┣ 10 (0x7f74576f5fb5fd9d9b1f3841019912e88b22d6ef) 0.000000 Ξ
┃┃┣ 11 (0x4d3693576411edae29a1dfe19dce8deb98f1460b) 0.000000 Ξ
┃┃┣ 12 (0x86c

TODO: talk about the light stuff here

In [ ]:
!bw2 i switch.ent
!bw2 bc -t switch.ent -u $NAMESPACE/s.switch/* -v

TODO: talk about the switch stuff here

**Here, create the light widget and have people publish to it using their own personal client**

Now, we can create a Python client with our entity and connect to our local agent.

We initialize the client with the entity file we want to use. We specify the entity file (and not our alias or VK) because the file contains the private signing key (SK) that we need for the agent to create proofs and sign our interactions with BOSSWAVE resources.

In [ ]:
from client import get_client
c = get_client(entity='myentity.ent')
import os
namespace = os.environ["NAMESPACE"]
print namespace
c.publish(namespace+"/s.light/light0/slot/state", (64,0,0,1), "True")

### Writing our service

In BOSSWAVE, each process, device and individual has its own key. Because permissions are managed at the level of keys, this gives us fine-grained control over what can talk or listen to what.

To begin, we create a new entity for our service binding the switch to the light.

In [4]:
!bw2 mke -o service.ent
!bw2 mkdot -f myentity.ent -t service.ent -u $NAMESPACE/s.light/light0/i.boolean/slot/state -x P
!bw2 mkdot -f myentity.ent -t service.ent -u $NAMESPACE/s.switch/switch0/i.boolean/signal/state -x C

 ╔╡127.0.0.1:28589 2.7.6 'Klystron'
 ╚╡peers=8 block=3100531 age=24s
Entity created
Public VK: FLlFdbqxmbEBDbB5CuJmyn9TcgRKV8ckuHaE-2_83Dw=
wrote key to file service.ent
Waiting for 1 object(s) to publish
Current BCIP set to 2 confirmation blocks or 20 block timeout
confirming: (last block genesis was 34 seconds ago)  
confirming:🔗🔗 (last block genesis was 7 seconds ago)  
confirming:🔗🔗 (last block genesis was 17 seconds ago)  
confirming:🔗🔗🔗🔗 (last block genesis was 7 seconds ago)  
confirming:🔗🔗🔗🔗 (last block genesis was 17 seconds ago)  
Successfully published Entity FLlFdbqxmbEBDbB5CuJmyn9TcgRKV8ckuHaE-2_83Dw=

All objects published
 ╔╡127.0.0.1:28589 2.7.6 'Klystron'
 ╚╡peers=7 block=3100535 age=20s
could not create dot: [419] Cannot grant dot, destination VK state: Unknown
 ╔╡127.0.0.1:28589 2.7.6 'Klystron'
 ╚╡peers=7 block=3100535 age=20s
could not create dot: [419] Cannot grant dot, destination VK state: Unknown


In [ ]:
from client import get_client
c = get_client(entity='myservice.ent')
import os
namespace = os.environ["NAMESPACE"]

In [ ]:
def mycallback(msg):
    print msg.uri, msg.payload
c.subscribe(namespace+"/*", mycallback)

We now give you a light, which already subscribes to its URL. We also provides an event log textbox that shows all commands published to that URL. The lastest event shows at the top of the textbox.

In [13]:
from widgets import Light
light = Light("scratch.ns")
light.display()

A Jupyter Widget

You can publish a "true" string to the light's URL to turn it on.

In [17]:
from client import get_client
c = get_client(entity='/etc/rise_entity.ent')

# Turn the light on
c.publish(light.url(), (64,0,0,1), "true")

You can publish a "false" string to the light's URL to turn it off.

In [18]:
# Turn the light off
c.publish(light.url(), (64,0,0,1), "false")

We now give you a switch, which publishes to its URL when clicked, and a event log that shows all commands sent to that URL. You can click on each button to issue an on or off command.

In [19]:
from widgets import Switch
switch = Switch()
switch.display()

A Jupyter Widget

Now, you can wire the switch to the light using their URLs.

In [20]:
import client

c = client.get_client()
def cb(msg):
    if msg.payload == "true":
        c.publish(light.url(), (64,0,0,1), "true")
    elif msg.payload == "false":
        c.publish(light.url(), (64,0,0,1), "false")

c.subscribe(switch.url(), cb)

AttributeError: Switch instance has no attribute '_url'